In [1]:
import h5py
import datetime
import json
import os
from pathlib import Path

In [2]:
def create_group_to_fill(TYPE, where, name):
    group=where.create_group(name)
    group.attrs["NX_class"]=TYPE
    return group

In [3]:
def write_data(dati, where):
    for row in dati:
        if (isinstance(dati[row], str) and row != "m_def") or isinstance(dati[row], (int, float, bool)):
            where.create_dataset(row, data=dati[row])
        elif isinstance(dati[row], dict):
            if dati[row].keys() == {"value", "unit", "direction"}:
                where.create_dataset(row, data=dati[row]["value"])
                where[row].attrs["units"]=dati[row]["unit"]
                where[row].attrs["direction"] = dati[row]["direction"]
            elif dati[row].keys() <= {"value", "unit"}:
                where.create_dataset(row, data=dati[row]["value"])
                where[row].attrs["units"]=dati[row]["unit"]
            elif "m_def" in dati[row].keys():
                newwhere=create_group_to_fill(dati[row]["m_def"], where, row)
                write_data(dati[row], newwhere)

In [4]:
def write_from_json(json_input, where):
    with open(json_input, "r", encoding="utf-8") as file:
        dati = json.load(file)
        write_data(dati, where)

In [5]:
def aprire_jsons(directory, entry):
    for file in os.listdir(directory):
        if Path(file).suffix == ".json" and "entry" in file:
            write_from_json(os.path.join(directory, file), entry)

    for file in os.listdir(directory):
        if Path(file).suffix == ".json" and "sample" in file:
            sample=create_group_to_fill("NXsample", entry, "sample")
            write_from_json(os.path.join(directory, file), sample)
        if Path(file).suffix == ".json" and "user" in file:
            user=create_group_to_fill("NXuser", entry, "user")
            write_from_json(os.path.join(directory, file), user)
        if Path(file).suffix == ".json" and "instrument" in file:
            instr=create_group_to_fill("NXinstrument", entry, "instrument")
            write_from_json(os.path.join(directory, file), instr)
            

In [6]:
with h5py.File("afm_file_auto1.nxs", "w") as f:
    entry = f.create_group("entry")
    entry.attrs["NX_class"] = "NXentry"
    entry.create_dataset("definition", data="NXafm")
    entry["definition"].attrs["version"] = "v2024.02"
    entry["definition"].attrs["URL"] = "https://github.com/FAIRmat-NFDI/nexus_definitions/blob/fairmat/contributed_definitions/NXafm.nxdl.xml"
    aprire_jsons("./jsons_for_afm/" , entry)
    #### Completiamo generando i link necessari
    scan = entry["instrument"].create_group ("scan_environment")
    scan.attrs["NX_class"] = "NXenvironment"
    scan["height_piezo_sensor"] = entry["instrument"]["height_piezo_sensor"]
    scan["XY_piezo_sensor"] = entry["instrument"]["XY_piezo_sensor"]
    scan["tip_temp_sensor"] = entry["instrument"]["tip_temp_sensor"]
    scan.create_dataset("tip_temp", data=25)
    scan["tip_temp"].attrs["units"] = "celsius"
    #### Reproducibility indicators
    reprind = entry.create_group("reproducibility_indicators")
    reprind.attrs["NX_class"] = "NXcollection"
    reprind["cantilever_oscillator"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]
    reprind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    #### Resolution indicators
    resind = entry.create_group("resolution_indicators")
    resind.attrs["NX_class"] = "NXcollection"
    resind["oscillator_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["oscillator_excitation"]
    resind["cantilever_tip_temp"] = entry["instrument"]["scan_environment"]["tip_temp"]
    resind["amplitude_excitation"] = entry["instrument"]["cantilever"]["cantilever_oscillator"]["phase_lock_loop"]["amplitude_excitation"]

In [12]:
import json
from PIL import Image, TiffTags
from fractions import Fraction

def convert_to_serializable(obj):
    """Converte tipi non serializzabili in formati compatibili JSON."""
    if isinstance(obj, bytes):
        try:
            return obj.decode("utf-8", errors="replace")
        except Exception:
            return list(obj)  # come lista di int
    if isinstance(obj, (tuple, list)):
        return [convert_to_serializable(x) for x in obj]
    if isinstance(obj, dict):
        return {str(k): convert_to_serializable(v) for k, v in obj.items()}
    if isinstance(obj, Fraction):  # caso raro
        return float(obj)
    if hasattr(obj, "numerator") and hasattr(obj, "denominator"):
        # caso IFDRational o simile
        try:
            return float(obj)
        except Exception:
            return str(obj)
    if isinstance(obj, (int, float, str, bool)) or obj is None:
        return obj
    return str(obj)  # fallback: stringa

def estrai_metadata_tiff(percorso_file):
    """Estrae tutti i metadata da un file TIFF, inclusi i tag personalizzati."""
    metadata = {}
    with Image.open(percorso_file) as img:
        metadata["format"] = img.format
        metadata["mode"] = img.mode
        metadata["size"] = img.size

        if hasattr(img, "tag_v2"):
            tags = {}
            for k, v in img.tag_v2.items():
                tag_name = TiffTags.TAGS_V2.get(k, k)
                try:
                    hash(tag_name)
                except TypeError:
                    tag_name = str(tag_name)
                tags[str(tag_name)] = convert_to_serializable(v)
            metadata["tiff_tags"] = tags

    return metadata

if __name__ == "__main__":
    file_path = "./tiffs_for_sem/.tif"
    info = estrai_metadata_tiff(file_path)

    # Salva in JSON senza errori
    with open("metadata_tiff.json", "w", encoding="utf-8") as f:
        json.dump(info, f, indent=2, ensure_ascii=False)

    print(f"Metadata salvati in metadata_tiff.json con {len(info.get('tiff_tags', {}))} tag.")

    file_path = "./tiffs_for_sem/U1502_metal_cross_5um_015.tif"
    info = estrai_metadata_tiff(file_path)
    with open("metadata_tiff.json", "w", encoding="utf-8") as f:
        json.dump(info, f, indent=2, ensure_ascii=False)

    print(f"Metadata salvati in metadata_tiff.json con {len(info.get('tiff_tags', {}))} tag.")

Metadata salvati in metadata_tiff.json con 15 tag.


In [48]:
with h5py.File("sem_file_auto.nxs", "w") as f:
    entry = f.create_group("entry")
    entry.attrs["NX_class"] = "NXentry"
    entry.create_dataset("definition", data="NXem")
    entry["definition"].attrs["version"] = "v2024.02"
    entry["definition"].attrs["URL"] = "https://github.com/FAIRmat-NFDI/nexus_definitions/blob/a85e10cd0289f4e44b0fec011ff54703e6705383/contributed_definitions/NXem.nxdl.xml"
    entry.create_dataset("experiment_identifier", data="experiment_id")
    aprire_jsons("./jsons_for_sem/", entry)
    

In [49]:
from nexusformat.nexus import *

test = nxload("sem_file_auto.nxs")
print(test.tree)

root:NXroot
  entry:NXentry
    definition = 'NXem'
      @URL = 'https://github.com/FAIRmat-NFDI/nexus_definiti...'
      @version = 'v2024.02'
    experiment_identifier = 'experiment_id'
    sample:NXsample
      is_simulation = True
      name = 'Sample SEM'
      physical_form = 'bulk'
    user:NXuser
      ORCID = '0009-0004-3810-1808'
      address = 'Povo(TN)'
      affiliation = 'FBK'
      email = 'mbontorno@fbk.eu'
      facility_user_id = 'MB'
      name = 'Matteo Bontorno'
      role = 'Collaborator'


In [ ]:
class CharacterizationEquipmentEntryPoint(SchemaPackageEntryPoint):
    def load(self):
        from fabrication_facilities.schema_packages.equipments.character_equipment import (
            m_package,
        )

        return m_package
   